# Models

In this section, we’ll take a closer look at creating and using models. We’ll use the AutoModel class, which is handy when you want to instantiate any model from a checkpoint.

## Creating a Transformer
Let’s begin by examining what happens when we instantiate an AutoModel:

In [1]:
from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")

Similar to the tokenizer, the from_pretrained() method will download and cache the model data from the Hugging Face Hub. As mentioned previously, the checkpoint name corresponds to a specific model architecture and weights, in this case a BERT model with a basic architecture (12 layers, 768 hidden size, 12 attention heads) and cased inputs (meaning that the uppercase/lowercase distinction is important). There are many checkpoints available on the Hub — you can explore them here.

The AutoModel class and its associates are actually simple wrappers designed to fetch the appropriate model architecture for a given checkpoint. It’s an “auto” class meaning it will guess the appropriate model architecture for you and instantiate the correct model class. However, if you know the type of model you want to use, you can use the class that defines its architecture directly:

In [2]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")

## Loading and saving
Saving a model is as simple as saving a tokenizer. In fact, the models actually have the same save_pretrained() method, which saves the model’s weights and architecture configuration:

In [3]:
model.save_pretrained("lesson3")

This will save two files to your disk:


    ls directory_on_my_computer

    config.json model.safetensors

If you look inside the config.json file, you’ll see all the necessary attributes needed to build the model architecture. This file also contains some metadata, such as where the checkpoint originated and what 🤗 Transformers version you were using when you last saved the checkpoint.

The pytorch_model.safetensors file is known as the state dictionary; it contains all your model’s weights. The two files work together: the configuration file is needed to know about the model architecture, while the model weights are the parameters of the model.

To reuse a saved model, use the from_pretrained() method again:


In [5]:
from transformers import AutoModel

model = AutoModel.from_pretrained("hidden/lesson3")

A wonderful feature of the 🤗 Transformers library is the ability to easily share models and tokenizers with the community. To do this, make sure you have an account on Hugging Face. If you’re using a notebook, you can easily log in with this:

In [6]:
# only run if you don't already have
#!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 2.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]


In [10]:
# This widget did not load for me so I used an alternate method
from huggingface_hub import notebook_login

notebook_login()

In [12]:
#alternate method
from huggingface_hub import login

#enter your token here between () and make sure to delete after running
#and before pushing up

login(token="")


In [14]:
#verify your login

from huggingface_hub import whoami
whoami()

{'type': 'user',
 'id': '6962ca21d8c2183ea12213c9',
 'name': 'a-reusche92',
 'fullname': 'Andrew Reusche',
 'email': 'andrew.j.reusche@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'billingMode': 'prepaid',
 'periodEnd': 1769904000,
 'isPro': False,
 'avatarUrl': '/avatars/8c92179be8b400575337a5c5476e34ed.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Jupyter-push',
   'role': 'write',
   'createdAt': '2026-01-15T00:46:55.804Z'}}}

Otherwise, at your terminal run:

    huggingface-cli login

Then you can push the model to the Hub with the push_to_hub() method:

In [15]:
model.push_to_hub("example_my-awesome-model")

Processing Files (0 / 0): |                                        |  0.00B /  0.00B            
Processing Files (1 / 1): 100%|████████████████████████████████████|  433MB /  433MB,  310MB/s  
New Data Upload: |                                                 |  0.00B /  0.00B,  0.00B/s  


CommitInfo(commit_url='https://huggingface.co/a-reusche92/example_my-awesome-model/commit/b13d8c0dc0a459377d4dedc17867a23529e95283', commit_message='Upload model', commit_description='', oid='b13d8c0dc0a459377d4dedc17867a23529e95283', pr_url=None, repo_url=RepoUrl('https://huggingface.co/a-reusche92/example_my-awesome-model', endpoint='https://huggingface.co', repo_type='model', repo_id='a-reusche92/example_my-awesome-model'), pr_revision=None, pr_num=None)

In [18]:
#to fix future widgets
!pip install -U ipywidgets
!jupyter nbextension enable --py widgetsnbextension


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
